<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%208/Regresi%C3%B3n_lineal_vs_Redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este notebook se utilizará un dataset para regresión, con la finalidad de comprar el desempeño entre una regresión lineal y un par de redes neuronales. 

Se verá la importación de las depencias, la carga de los datos, el uso de [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) junto a [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) para conectar facilmente al procesamiento de los datos con el algoritmo (en este caso, regresión lineal). 

También, se verá la creación de las redes neuronales y el uso de callbacks (en particular, guardar checkpoints) durante el entrenamiento.

Para todos los modelos se evaluará el desempeño que se obtuvo y se harán comparaciones entre los tres.

# Dependencias

In [ ]:
!pip install -U plotly

In [ ]:
import os

# Manejo de los datos
import pandas as pd
import numpy as np

# Utilidades, modelo y métricas
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Para la red neuronal
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# Visualización
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
pio.templates.default = "plotly_white"

In [ ]:
# directorio para guardar los checkpoints de las redes neuronales
checkpoint_filepath = 'checkpoints'

# Funciones

In [ ]:
def plot_model_evaluation(real_train, predecido_train, real_test, predecido_test, plot_title):
    
    fig = make_subplots(
        rows = 1, cols = 2, 
        subplot_titles = ['Conjunto de entrenamiento', 'Conjunto de prueba'],        
    )

    fig.add_trace(
        go.Scatter(
            x = real_train, 
            y = predecido_train, 
            mode='markers', 
            ), 
        row = 1, col = 1
    )

    fig.add_shape(
        type = 'line', 
        x0 = np.min(predecido_train), y0 = np.min(predecido_train), 
        x1 = np.max(predecido_train), y1 = np.max(predecido_train), 
        line=dict(dash = 'dot')
    )

    fig.add_trace(
        go.Scatter(x = real_test, y = predecido_test, mode = 'markers'), 
        row = 1, col = 2
    )

    fig.add_shape(
        type = 'line', 
        x0 = np.min(predecido_test), y0 = np.min(predecido_test), 
        x1 = np.max(predecido_test), y1 = np.max(predecido_test), 
        line=dict(dash = 'dot'), 
        row = 1, col = 2
    )
    
    fig.update_layout(
        showlegend=False, 
        title_text=plot_title
    )

    fig.update_xaxes(title_text = 'Cargos reales', row = 1, col = 1)
    fig.update_xaxes(title_text = 'Cargos reales', row = 1, col = 2)
    fig.update_yaxes(title_text = 'Cargos predecidos', row = 1, col = 1)
    fig.update_yaxes(title_text = 'Cargos predecidos', row = 1, col = 2)

    return fig


# Datos

Se trabajaran con los datos de [Medical Cost Personal Datasets](https://www.kaggle.com/mirichoi0218/insurance). Contiene 7 columnas descritas a continuación:



    age: age of primary beneficiary

    sex: insurance contractor gender, female, male

    bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
    objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

    children: Number of children covered by health insurance / Number of dependents

    smoker: Smoking

    region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

    charges: Individual medical costs billed by health insurance


Se quiere saber si se pueden predecir los cargos (charges) en función de las demás variables.

## Carga de los datos

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Descripción de los datos

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


##**Ejercicio**: Realizar un análisis descriptivo de los datos.

## División en conjuntos de entrenamiento y prueba

Separamos el DataFrame en dos: 
1. X: Contiene las variables regresoras.
2. y: La variable de respuesta.

In [ ]:
X = data.drop(columns = 'charges')
y = data['charges']

Ahora divimos el dataset en el conjunto de entrenamiento y de prueba. Esto, como hemos visto, nos ayuda a evaluar si el modelo "aprendió" a generalizar las predicciones o si se sobreajustó.

El conjunto de entrenamiento tendrá el 70% de los datos totales.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .7, random_state = 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(936, 6) (936,) (402, 6) (402,)


# Regressión Lineal

Primero veremos los resultados que obtenemos con una regresión lineal (múltiple). Sin entrar en detalle a los supuestos del algoritmo, la hipótesis es la siguiente.

\begin{align}
    y_i = \beta_0 + \beta_1x_{i1} + \dots + \beta_px_{ip} + \epsilon_i,
\end{align}

donde, para el *i-ésimo* caso, $y_i$ es la variable de respuesta, $x_{ij}$ es la medición de la *j-ésima* variable, $b_j$ es el coeficiente de ésta variable y $\epsilon_i$ es el *error* (que queremos que sea lo más pequeño posible). 

Es decir, **la variable de respuesta es una *combinación lineal* de los regresores más un error**. Equivalentemente, se encuentra el hiperplano que 
"mejor se ajuste" a los datos cuando se minimiza el error.

## Algoritmo/Arquitectura

Crearemos un [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para facilitar el uso del modelo para las predicciones. 

El primer paso es un tranformador de columnas que se va a encargar de crear las variables dummies de las variables categóricas.

El segundo paso, la regresión lineal.

In [ ]:
transformer = ColumnTransformer(
    [
     ("dummies", OneHotEncoder(drop = 'first'), ['sex', 'smoker', 'region'])
    ], 
    remainder = 'passthrough'
)

lr_pipe = Pipeline(
    [
     ("transformador", transformer), 
     ('linear_regression', LinearRegression())
    ]
)

## Entrenamiento

Usamos la función ```fit``` para ajustar el modelo a nuestros datos.

In [ ]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('transformador',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummies',
                                                  OneHotEncoder(drop='first'),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('linear_regression', LinearRegression())])

## Evaluación

Con el modelo entrenado, usamos la función ```predict``` para obtener las predicciones del modelo con los datos que le suministramos.

Obtenemos las predicciones de ambos conjuntos de datos (entrenamiento y prueba).

In [ ]:
y_pred_train_lr = lr_pipe.predict(X_train)
y_pred_test_lr = lr_pipe.predict(X_test)

### Métricas

Las métricas que usaremos son 3:

1. **Coeficiente de determinación ($R^2$)**: El mejor posible valor es 1 (toda la variabilidad de la variable dependiente se puede explicar con las variables independientes). Un modelo que siempre predice la media de la variable de respuesta tiene un resultado de 0. Modelos con peor desempeño que el que sólo predice la media, obtienen valores negativos.

2. **Error cuadrático promedio (MSE)**: El promedio de los errores al cuadrado.

3. **Error absoluto promedio (MSE)**: El promedio de los valores absolutos de los errores.

In [ ]:
print(
    f"R2 train: {r2_score(y_train, y_pred_train_lr)}", 
    f"R2 test: {r2_score(y_test, y_pred_test_lr)}", 
    "",
    f"MSE train: {mean_squared_error(y_train, y_pred_train_lr)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_lr)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_lr)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_lr)}", 
    sep = '\n'
)

R2 train: 0.7618780003618578
R2 test: 0.7166124432331722

MSE train: 36576361.89065711
MSE test: 36800107.88801561

MAE train: 4175.618401731288
MAE test: 4226.647664219914


El coeficiente de determinación de .76 en entramiento y .71 en prueba nos indica que el modelo se comporta de manera similar con datos vistos durante el entramiento y fuera de estos (aparentemente no hay sobreajuste).

### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_lr, y_test, y_pred_test_lr, "Regresión lineal")

En la gráfica se observan 3 grupos distintos de puntos, esto nos indica dos cosas: 

1. Al menos un supuesto del algoritmo no se cumple.
2. Existen 3 o 4 grupos en los datos. Posiblemente con ingeniería de características se pueda mejorar el resultado.

# Primer Red Neuronal

La arquitectura de la primera red neuronal será muy similar a una regresión lineal, la única diferencia radica en el algoritmo de optimización de la función de pérdida.

## Preparación de los datos

Ocuparemos el mismo transformador que se creo y ajustó cuando usamos la regresión lineal. Los datos *preparados* nos van a servir para ambas redes neuronales.

In [ ]:
#X_train_transformed
X_train_tr = transformer.transform(X_train)
X_test_tr = transformer.transform(X_test)

## Algoritmo/arquitectura

La arquitectura sólo consiste en una capa de entrada y una de salida con activación lineal. 

Al inicio ponemos un par de semillas para asegurarnos que los resultados se puedan replicar.

In [ ]:
np.random.seed(10)
tf.random.set_seed(10)

model = keras.Sequential(
    [
     layers.Input([X_train_tr.shape[1]]),
    #  layers.LayerNormalization(input_shape = [X_train_tr.shape[1]]),
     layers.Dense(1, activation='linear')
    ]
)

Compilamos el modelo indicando que el optimizador es RMSprop, la función de pérdida el error cuadrático medio e indicamos que monitoreé el error absoluto medio.

In [ ]:
model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=.1), 
    loss = 'mse', 
    metrics = ['mae'], 
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


Definimos el número de épocas, la ruta donde se deben guardar los checkpoints y creamos el callback que se encargará de guardar los checkpoints.

In [ ]:
epochs = 200

weights_filepath = os.path.join(checkpoint_filepath, 'model_1')

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = weights_filepath, 
    save_best_only = True, 
    save_weights_only = True, 
    monitor = 'val_mae', 
    mode = 'min'
)

El callback para los checkpoints sólo guardará los pesos (```save_weights_only = True```) del mejor modelo (```save_best_only = True```) considerando el error absoluto medio del conjunto de validación (```monitor = 'val_mae'```) más pequeño (```mode = 'min'```) hasta el momento.

## Entrenamiento

Ya que esta definida la arquitectura del modelo, se compilo y se crearon los callbacks necesarios, usamos la función ```fit``` para entrenar la red neuronal.

In [ ]:
history = model.fit(
    X_train_tr, y_train, 
    epochs=epochs, 
    validation_split=.2, 
    callbacks=[model_checkpoint_callback]
    )

Epoch 1/200
24/24 [==============================] - 1s 9ms/step - loss: 337746464.0000 - mae: 13571.1084 - val_loss: 298195872.0000 - val_mae: 12156.6143
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 332252864.0000 - mae: 13382.3047 - val_loss: 293513088.0000 - val_mae: 11980.5703
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 327301600.0000 - mae: 13207.8291 - val_loss: 288939232.0000 - val_mae: 11806.2158
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 322428896.0000 - mae: 13035.0986 - val_loss: 284512512.0000 - val_mae: 11635.0654
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 317614176.0000 - mae: 12863.3105 - val_loss: 280186688.0000 - val_mae: 11465.4971
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 312897440.0000 - mae: 12691.9795 - val_loss: 275836704.0000 - val_mae: 11292.5635
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - los

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['mae'], name = 'entrenamiento')
)

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['val_mae'], name = 'validación')
)

fig.update_layout(
    hovermode = 'x unified', 
    legend_title = 'Conjunto de', 
    xaxis_title = 'Época', 
    yaxis_title = 'Error Absoluto Medio (MAE)', 
    title = 'MAE en cada época del entrenamiento<br><sup>Primera red neuronal</sup>', 
)


La gráfica muestra que el MAE llegó a un mínimo alrededor de la época 40 y luego el desempeño empeoró. Gracias a que agregamos el callback para guardar únicamente el mejor modelo basado en esta métrica, basta que lo carguemos con la función ```load_weights```.

In [ ]:
model.load_weights(weights_filepath)

## Evaluación

Usamos la función ```predict``` para predecir usando el modelo que acabamos de entrenar.

In [ ]:
y_pred_train_nn1 = model.predict(X_train_tr)[:, 0]
y_pred_test_nn1 = model.predict(X_test_tr)[:, 0]

### Métricas

In [ ]:
print(
    f"R2 train: {r2_score(y_train, y_pred_train_nn1)}", 
    f"R2 test: {r2_score(y_test, y_pred_test_nn1)}", 
    "",
    f"MSE train: {mean_squared_error(y_train, y_pred_train_nn1)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_nn1)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_nn1)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_nn1)}", 
    sep = '\n'
)

R2 train: -0.11405321987258676
R2 test: -0.13159156212284318

MSE train: 171122423.78878692
MSE test: 146946083.4709567

MAE train: 7761.346792911993
MAE test: 7114.495580171448


En esta ocasión, el coeficiente de determinación tuvo valores negativos, lo que nos indica que el modelo no sirve para predecir la variable de respuesta.

### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_nn1, y_test, y_pred_test_nn1, 'Primera red neuronal')

Visualmente, se confirma lo que la $R^2$ nos indicó, se observan tres grupos de datos, y la red sólo aprendio a "predecir correctamente" uno de ellos.

Crearemos otra red con más capas esperando que se comporte mejor.

# Segunda Red Neuronal

## Algoritmo/Aquitectura

Para esta segunda red neuronal, estamos agregando 3 capas densas con 64 neuronas cada una, es decir, el modelo va a tener 5 capas en total:

1. Una capa de entrada,
2. tres capas ocultas, y
3. una capa de salida.

In [ ]:
np.random.seed(10)
tf.random.set_seed(10)

model = keras.Sequential(
    [
     layers.Input([X_train_tr.shape[1]]),
     layers.Dense(64, activation='relu'),
     layers.Dense(64, activation='relu'),
     layers.Dense(64, activation='relu'),     
     layers.Dense(1)
    ]
)

In [ ]:
model.compile(
    optimizer = keras.optimizers.RMSprop(.01), 
    loss = 'mse', 
    metrics = ['mae']
)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                576       
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,961
Trainable params: 8,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 200

weights_filepath = os.path.join(checkpoint_filepath, 'model_2')

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = weights_filepath, 
    save_best_only = True, 
    save_weights_only = True, 
    monitor = 'val_mae', 
    mode = 'min'
)


## Entrenamiento

In [ ]:

history = model.fit(
    X_train_tr, y_train, 
    epochs=epochs, 
    validation_split=.2, 
    callbacks=[model_checkpoint_callback]
    )

Epoch 1/200
24/24 [==============================] - 1s 10ms/step - loss: 198219648.0000 - mae: 10151.2754 - val_loss: 126943272.0000 - val_mae: 9164.9443
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 137418176.0000 - mae: 9418.1621 - val_loss: 125209984.0000 - val_mae: 8882.0654
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 135042752.0000 - mae: 9360.1279 - val_loss: 127599880.0000 - val_mae: 9636.9570
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 133094928.0000 - mae: 9307.5537 - val_loss: 132495056.0000 - val_mae: 10201.2314
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 132692672.0000 - mae: 9320.8213 - val_loss: 122364024.0000 - val_mae: 8005.9160
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 129281776.0000 - mae: 9185.5205 - val_loss: 122406576.0000 - val_mae: 7654.8013
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 127561

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['mae'], name = 'entrenamiento')
)

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['val_mae'], name = 'validación')
)

fig.update_layout(
    hovermode = 'x unified', 
    legend_title = 'Conjunto de', 
    xaxis_title = 'Época', 
    yaxis_title = 'Error Absoluto Medio (MAE)', 
    title = 'MAE en cada época del entrenamiento<br><sup>Segunda red neuronal</sup>', 
)


Observamos un comportamiento "errático" en los valores del MAE a lo largo de cada época. Lo importante es que la gráfica muestra que el modelo sí aprendió (el valor de la métrica fue disminuyendo conforme avanzan las épocas de entrenamiento).

In [ ]:
model.load_weights(weights_filepath)

## Evaluación

In [ ]:
y_pred_train_nn2 = model.predict(X_train_tr)[:, 0]
y_pred_test_nn2 = model.predict(X_test_tr)[:, 0]

### Métricas

In [ ]:
print(
    f"R2 train: {r2_score(y_train, y_pred_train_nn2)}", 
    f"R2 test: {r2_score(y_test, y_pred_test_nn2)}", 
    "",
    f"MSE train: {mean_squared_error(y_train, y_pred_train_nn2)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_nn2)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_nn2)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_nn2)}", 
    sep = '\n'
)

R2 train: 0.848485778204348
R2 test: 0.8240586212831948

MSE train: 23273107.971546568
MSE test: 22847374.7846029

MAE train: 2295.9363390972435
MAE test: 2408.6658949440202


Obtenemos una $R^2$ mejor que cuando ajustamos el modelo con la regresión lineal y el MAE se redujo casi a la mitad. Parece que esta red aprendió mejor a predecir los datos.

### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_nn2, y_test, y_pred_test_nn2, 'Segunda red neuronal')

Nuevamente, parece que hay 3 o 4 grupos de casos. Sin embargo, la diferencia es menos obvia que cuando se graficaron los resultados de la regresión lineal (y más aún en comparación a la primera red). 

# Conclusión

La regresión lineal fue un buen primer acercamiento a este problema considerando que arrojó una $R^2$ mayor a 0.7. Sin embargo, para cargos grandes no se comportó bien y se vieron 3 o 4 grupos distintos. Un análisis a detalle de las variables e ingeniería de características seguramente ayuda a mejorar el desempeño. 

El segundo intento con la red neuronal predice mejor que la anterior, es decir, logró *aprender* mejor las relaciones entre las variables. Con más capas, el desempeño mejora, pero se dejará como ejercicio.

# Ejercicios

1. Realizar un análisis descriptivo de los datos.

2. ¿Por qué se obtuvieron mejores resultados con la red neuronal que con la regresión lineal?

    2.1 ¿Por qué la primera red neuronal predice peor que la regresión lineal?
    
    2.2 ¿Cuál es la intuición detrás de "si hay más capas, usualmente la red neuronal predice mejor"?

3. Aplicar transformaciones a las variables y ajustar de nuevo la regresión lineal y una red neuronal para ver si mejoran los resultados.

4. Diseñar una red más compleja que mejore los resultados obtenidos

# Extra

1. Comprobar los supuestos de la regresión lineal:

    1. Linearidad.

    2. Los errores son ruido blanco.

    3. Sin multicolinealidad en los predictores

    4. Homocedasticidad

2. ¿Se viola alguno? ¿Se puede hacer algo para arreglarlo?

3. El problema que se presentó es un problema de **regresión**. Además de la regresión lineal y la red neuronal presentada, ¿qué otros algoritmos de regresión conoce?

    1. Utilice al menos uno, evalué y compare los resultados obtenidos.

    2. ¿Los resultados fueron mejores que con la regresión lineal? En caso de ser afirmativa la respuesta, ¿nos dice algo sobre los datos?